In [1]:
index = 2239

g_S2_path = r"D:\Code\modis-s1-s2\output\generated_image\g_S2_" + str(index) + ".npy"
real_S2_path = r"D:\Code\modis-s1-s2\output\generated_image\real_S2_" + str(index) + ".npy"
before_S2_path = r"D:\Code\modis-s1-s2\output\generated_image\before_S2_" + str(index) + ".npy"
after_S2_path = r"D:\Code\modis-s1-s2\output\generated_image\after_S2_" + str(index) + ".npy"

g_S2_tif_path = r"D:\Code\modis-s1-s2\output\generated_image\g_S2_" + str(index) + ".tif"
real_S2_tif_path = r"D:\Code\modis-s1-s2\output\generated_image\real_S2_" + str(index) + ".tif"
before_S2_tif_path = r"D:\Code\modis-s1-s2\output\generated_image\before_S2_" + str(index) + ".tif"
after_S2_tif_path = r"D:\Code\modis-s1-s2\output\generated_image\after_S2_" + str(index) + ".tif"

tif_path = r"D:\ENVI\data\NingBo\NB_roi\cropped_data\test\S2\S2_" + str(index) + ".tif"
model_path = r"D:\Code\modis-s1-s2\model\cGAN-S1\wgan_generator_epoch_300.pth"

In [5]:
import torch
import numpy as np
import sys
sys.path.append(r"D:\Code\modis-s1-s2\src")
# sys.path.append(r"C:\Users\Administrator\Desktop\kaggle\src-3")
from model import Generator
from load_dataset import get_dataset

def generate_image(generator, MODIS, S1, before, after, save_path=None):
    generator.eval()
    with torch.no_grad():
        g_S2 = generator(MODIS, S1, before, after).cpu().numpy()
    g_S2 = np.squeeze(g_S2)
    g_S2 = (g_S2 + 1) / 2
    g_S2 = (10000 - 0) * g_S2 + 0
    g_S2 = np.clip(g_S2, 0, 10000)

    if save_path is not None:
        np.save(save_path, g_S2)
    return g_S2

coupled_data = get_dataset()[2][index-2194]
MODIS_image, S1_image, S2_image, before_image, after_image = coupled_data
MODIS_image = MODIS_image.reshape((1, 6, 5, 5)).cuda()
S1_image = S1_image.reshape((1, 2, 250, 250)).cuda()
real_S2_image = S2_image.reshape((1, 8, 250, 250)).cuda()
before_image = before_image.reshape((1, 8, 250, 250)).cuda()
after_image = after_image.reshape((1, 8, 250, 250)).cuda()
generator = torch.load(model_path)
g_S2 = generate_image(generator, MODIS_image, S1_image, before_image, after_image, g_S2_path)

print(g_S2.shape)

(8, 250, 250)


In [3]:
import numpy as np

def denormalization(norm_s2):
    norm_s2 = (10000 - 0) * norm_s2 + 0
    s2 = np.clip(norm_s2, 0, 10000)
    return s2

real_s2 = np.load(r"D:\Code\modis-s1-s2\dataset\SatelliteImages\test\S2\S2_" + str(index) + ".npy")
before_s2 = np.load(r"D:\Code\modis-s1-s2\dataset\SatelliteImages\test\before\before_" + str(index) + ".npy")
after_s2 = np.load(r"D:\Code\modis-s1-s2\dataset\SatelliteImages\test\after\after_" + str(index) + ".npy")

np.save(real_S2_path, denormalization(real_s2))
np.save(before_S2_path, denormalization(before_s2))
np.save(after_S2_path, denormalization(after_s2))

In [2]:
from osgeo import gdal
import numpy as np

def save_numpy_as_geotiff(numpy_path, reference_tif_path, output_tif_path):
    numpy_array = np.load(numpy_path)

    # 读取参考tif文件获取地理信息
    reference_dataset = gdal.Open(reference_tif_path)
    reference_geotransform = reference_dataset.GetGeoTransform()
    reference_projection = reference_dataset.GetProjection()

    # 创建输出GeoTIFF文件
    driver = gdal.GetDriverByName("GTiff")
    num_bands, rows, cols = numpy_array.shape
    output_dataset = driver.Create(output_tif_path, cols, rows, num_bands, gdal.GDT_Int16)

    # 设置地理信息
    output_dataset.SetGeoTransform(reference_geotransform)
    output_dataset.SetProjection(reference_projection)

    # 写入数据
    for band_num in range(num_bands):
        output_band = output_dataset.GetRasterBand(band_num + 1)
        output_band.WriteArray(numpy_array[band_num, :, :])

    # 关闭数据集
    output_dataset = None
    reference_dataset = None

save_numpy_as_geotiff(g_S2_path, tif_path, g_S2_tif_path)
# save_numpy_as_geotiff(real_S2_path, tif_path, real_S2_tif_path)
# save_numpy_as_geotiff(before_S2_path, tif_path, before_S2_tif_path)
# save_numpy_as_geotiff(after_S2_path, tif_path, after_S2_tif_path)